In [61]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd

In [70]:
pmd_df = pd.read_csv(home_dir+"data/pmd/pmd_with_rsids.tsv", sep="\t")
print(pmd_df.shape)
print(pmd_df.columns)

pmd_df.drop_duplicates(keep="first", ignore_index=True, inplace=True)
print(pmd_df.shape) 

pmd_df = pmd_df[pmd_df["source"].apply(lambda x: True if "Human" in str(x) else False)] # only keeping human variants
print("after removing non-human entries:", pmd_df.shape)

(2744, 72)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'authors', 'journal', 'title',
       'medline', 'crossref', 'uniprot_id', 'ensembl_id', 'other_ref',
       'protein', 'source', 'expression_sys', 'mut_PMD', 'mut_real',
       'function', 'fb', 'structure', 'strB', 'stability', 'staB',
       'expression', 'eB', 'transport', 'tB', 'maturation', 'mB', 'disease',
       'dB', 'uni_real', 'uni_realid', 'uni_start', 'uni_finish', 'uniq_start',
       'uniq_finish', 'uni_loc', 'ens_real', 'ens_organism', 'ens_start',
       'ens_finish', 'ensq_start', 'ensq_finish', 'ens_loc', 'pos_real',
       'mt_real', 'taxid', 'wt', 'mut', 'prot_pos', 'function_summarized',
       'functional_effect', 'gene_symbol', 'gene_SP', 'description', 'seq',
       'snp_id', 'mrna_acc', 'mrna_ver', 'mrna_pos', 'allele', 'protein_acc',
       'protein_ver', 'verified', 'chrom', 'chrom_pos', 'variation',
       'variant_type', 'ref_allele', 'alt_allele', 'pmd_nr_id'],
      dtype='object')
(2657, 72)
after remov

In [71]:
# chrom_pos is 1-indexed
# snp_id is rs-id from NCBI
info_cols = ['mut_id', 'md5', 'pmd_id', 'nr', 'pmd_nr_id', 'crossref', 'uniprot_id', 'protein', 'source', 'mut_PMD', 'mut_real', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'function', 'function_summarized', 'functional_effect']
other_cols = ['seq']
columns = info_cols + other_cols
pmd_df = pmd_df[columns]
print(pmd_df.shape)
print(pmd_df.columns)

(2638, 23)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'pmd_nr_id', 'crossref', 'uniprot_id',
       'protein', 'source', 'mut_PMD', 'mut_real', 'wt', 'mut', 'prot_pos',
       'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'function',
       'function_summarized', 'functional_effect', 'seq'],
      dtype='object')


In [72]:
# filtering the dataset on the protein sequence len <=1022
sequences_df = pmd_df[['pmd_nr_id', 'crossref', 'seq']].drop_duplicates(keep="first")
print("#-of sequences b4 seq-len filter", sequences_df.shape[0])
protid_seq_tuple_list = [(tuple.pmd_nr_id, tuple.seq) for tuple in sequences_df.itertuples() if len(str(tuple.seq))<=1022]
print("#-of sequences after seq-len filter", len(protid_seq_tuple_list))

new_protein_acc_list = list(zip(*protid_seq_tuple_list))[0]
pmd_df = pmd_df[pmd_df["pmd_nr_id"].isin(new_protein_acc_list)]
print("after filtering on seq-len <=1022:", pmd_df.shape)

#-of sequences b4 seq-len filter 858
#-of sequences after seq-len filter 821
after filtering on seq-len <=1022: (2421, 23)


In [83]:
pmd_df[['mut_id', 'md5', 'pmd_id', 'pmd_nr_id', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'function', 'function_summarized', 'functional_effect']].value_counts()

mut_id   md5                               pmd_id   pmd_nr_id  wt  mut  prot_pos  chrom  chrom_pos   ref_allele  alt_allele  snp_id       function                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    function_summarized   functional_effect
113507   e

In [76]:
pmd_dbnsfp_df = pd.read_csv(home_dir+"data/pmd/pmd_with_dbnsfp.tsv", sep="\t")
pmd_dbnsfp_df.rename(columns={'#chr':'chrom', 'pos(1-based)':'chrom_pos', 'ref':'ref_allele', 'alt':'alt_allele', 'rs_dbSNP':'snp_id'}, inplace=True)
print(pmd_dbnsfp_df.shape)
print(pmd_dbnsfp_df.columns)
pmd_dbnsfp_df["genename"]

(7310, 35)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'pmd_nr_id', 'crossref', 'uniprot_id',
       'protein', 'source', 'mut_PMD', 'mut_real', 'wt', 'mut', 'prot_pos',
       'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'genename',
       'ensembl_id', 'function', 'function_summarized', 'functional_effect',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq'],
      dtype='object')


0                                                    MC4R
1                                                    MC4R
2                      BDH1;BDH1;BDH1;BDH1;BDH1;BDH1;BDH1
3                      BDH1;BDH1;BDH1;BDH1;BDH1;BDH1;BDH1
4                                     BDH1;BDH1;BDH1;BDH1
                              ...                        
7305    CDKN2A;CDKN2A;CDKN2A;CDKN2A;CDKN2A;CDKN2A;CDKN...
7306                        APEX1;APEX1;APEX1;APEX1;APEX1
7307    APEX1;APEX1;APEX1;APEX1;APEX1;APEX1;APEX1;APEX...
7308            APEX1;APEX1;APEX1;APEX1;APEX1;APEX1;APEX1
7309            APEX1;APEX1;APEX1;APEX1;APEX1;APEX1;APEX1
Name: genename, Length: 7310, dtype: object

In [84]:
pmd_dbnsfp_df[['mut_id', 'md5', 'pmd_id', 'pmd_nr_id', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 'snp_id', 'function', 'function_summarized', 'functional_effect']].value_counts()

mut_id   md5                               pmd_id   pmd_nr_id  wt  mut  prot_pos  chrom  chrom_pos  ref_allele  alt_allele  snp_id       function                                                                                                                                                                                                                                                                                                function_summarized                         functional_effect
113507   ebef07b70de3e26d617f874d57f3f95c  A013365  A013365_2  K   E    329       1      75761161   A           G           rs77931234    Enzyme activity at 31 C without [0] and with [-] GroESL chaperonins co-overexpression: Enzyme activity at 37 C without [0] and with [-] GroESL chaperonins co-overexpression                                                                                                                           ['0', '-', '0', '-']                        Knock-out            1
10324

In [81]:
columns = ['mut_id', 'md5', 'pmd_id', 'pmd_nr_id', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele']
x = pmd_df[['mut_id', 'md5', 'pmd_id', 'pmd_nr_id', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele']]
y = pmd_dbnsfp_df[['mut_id', 'md5', 'pmd_id', 'pmd_nr_id', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele']]
z = pd.merge(x, y, how='outer', on=columns)
# z.to_csv(home_dir+"data/pmd/pmd_x.tsv", sep="\t", index=False)
z.shape
z.value_counts()

mut_id   md5                               pmd_id   pmd_nr_id  wt  mut  prot_pos  chrom  chrom_pos    ref_allele  alt_allele
113507   ebef07b70de3e26d617f874d57f3f95c  A013365  A013365_2  K   E    329       1      75761161.0   A           C             1
1027546  77a6752e78a958ef63657837ceb1f66c  A910709  A910709_1  C   S    64        2      127421403.0  G           C             1
1028445  94eb709966e01fbc15dc1c46a0d03ac5  A910888  A910888_1  V   A    121       8      85474334.0   T           C             1
1028439  94eb709966e01fbc15dc1c46a0d03ac5  A910885  A910885_1  V   L    142       8      85474396.0   G           C             1
1028438  94eb709966e01fbc15dc1c46a0d03ac5  A910885  A910885_1  V   I    142       8      85474396.0   G           A             1
                                                                                                                               ..
931308   77a6752e78a958ef63657837ceb1f66c  A014435  A014435_1  R   W    394       2      127428

In [82]:
8536-7310

1226